In [10]:
import time
import requests
import pandas as pd

In [44]:
my_df = pd.read_csv('../out', sep = '\t', dtype=str)
dois = my_df['DOI'].dropna().unique()

In [ ]:
#display(my_df)
print(len(dois))

In [13]:
CROSSREF_URL = "https://api.crossref.org/v1/works/"
def get_crossref(doi):
    url = f"{CROSSREF_URL}{doi}"
    response = requests.get(url, timeout = 5)

    if response.status_code == 200:
        data = response.json().get("message", {})
        r = data.get("update-to", "Not retracted")
        print(data)
        if r != "Not retracted":
            if r[0]['type'] == 'retraction':
                print("Retracted")
                print(r)
        else:
            print('Not retracted')

In [14]:
#get_crossref('10.1128/MCB.00813-07')

In [33]:
test = dois[200:400]

In [35]:
import requests
import concurrent.futures
import pandas as pd
from tqdm import tqdm  

CROSSREF_URL = "https://api.crossref.org/v1/works/"

def get_crossref_metadata(doi):
    """Fetch metadata for a given DOI and check if it's retracted."""
    url = f"{CROSSREF_URL}{doi}"
    try:
        response = requests.get(url, timeout = 25)
        if response.status_code == 200:
            data = response.json().get("message", {})
            r = data.get("update-to", "Not retracted")
            if r != "Not retracted":
                if r[0].get('type') == 'retraction':
                    return "Retracted"
                else:
                    return r[0].get('type')
            else:
                return "Not retracted"
        else:
            return f"Error: {response.status_code}"
    except requests.exceptions.RequestException as e:
        return f"Request Error: {e}"

def check_dois_in_parallel(dois, batch_size=10, max_workers=1):
    """Processes DOIs in parallel and saves results in a new DataFrame column."""
   # dois = df[doi_column].tolist()
    results = {}

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_doi = {executor.submit(get_crossref_metadata, doi): doi for doi in dois}
        
        with tqdm(total=len(dois), desc="Checking Retractions", unit="DOI") as pbar:
            for future in concurrent.futures.as_completed(future_to_doi):
                doi = future_to_doi[future]
                status = future.result()
                results[doi] = status
                pbar.update(1)  # Update progress bar after each DOI

    #df["Retraction_Status"] = results  # Add results as a new column
    return results


status = check_dois_in_parallel(test)
#print(status)


Checking Retractions:   2%|▌                   | 5/200 [00:52<34:03, 10.48s/DOI]


KeyboardInterrupt: 

In [36]:
df = pd.read_csv('retraction_watch.csv',  dtype=str)
print(df.columns)

Index(['Record ID', 'Title', 'Subject', 'Institution', 'Journal', 'Publisher',
       'Country', 'Author', 'URLS', 'ArticleType', 'RetractionDate',
       'RetractionDOI', 'RetractionPubMedID', 'OriginalPaperDate',
       'OriginalPaperDOI', 'OriginalPaperPubMedID', 'RetractionNature',
       'Reason', 'Paywalled', 'Notes'],
      dtype='object')


In [32]:
print(df['Reason'][df['OriginalPaperDOI'] == '10.1016/j.biopha.2018.03.122'].to_string(index = False))

+Duplication of/in Image;+Investigation by Thir...


In [39]:
print(df['Reason'][10])
pd.set_option('display.max_colwidth', None)
my_df = pd.read_csv('Retracted_sample', sep = '\t', dtype=str)
status = my_df['Retracted DOI']
for i in status:
    a = df['OriginalPaperDOI'].isin([i])
    if not df[a].empty:
        print(i)
        print(df[a])
        print(df[a]['Reason'].to_string(index = False).split(';'))
        break

+Author Unresponsive;+Concerns/Issues about Referencing/Attributions;+Concerns/Issues with Peer Review;+Fake Peer Review;+Investigation by Journal/Publisher;+Unreliable Results and/or Conclusions;
10.1016/j.biopha.2018.03.122
      Record ID  \
18621     44005   

                                                                                     Title  \
18621  MicroRNA-361-5p inhibits papillary thyroid carcinoma progression by targeting ROCK1   

                                                               Subject  \
18621  (BLS) Biology - Cancer;(BLS) Biology - Cellular;(BLS) Genetics;   

                                                                                                                                                                                                                                                 Institution  \
18621  Department of Thyroid Surgery, The First Hospital of Jilin University, 71# Xinmin Street, Changchun 130021, China; Departments of Hepa

In [42]:
print(len(status))
#print(status.keys())
"""reason = {}
for key , value in status.items():
    if value == 'Retracted':
        reason[key] = value
res = sum(x == 'Retracted' for x in status.values())
print(res)
print(len(reason))
"""
retracted = {}
for i in status:
    a = df['OriginalPaperDOI'].isin([i])
    if not df[a].empty:
        retracted[i] = df[a]['Reason'].to_string(index = False)
        
"""for doi in reason.keys():
    a = df['OriginalPaperDOI'].isin([doi])
    if not df[a].empty:
        #reason.append(df[a]['Reason'].to_string(index = False))
        #retracted[doi].append(df[a]['Reason'].to_string(index = False))
        retracted[doi] = df[a]['Reason'].to_string(index = False)
            #break"""
print(retracted)
print(len(retracted))

73
{'10.1016/j.biopha.2018.03.122': '+Duplication of/in Image;+Investigation by Third Party;+Paper Mill;+Results Not Reproducible;', '10.1016/j.lfs.2020.117519': '+Concerns/Issues About Data;+Duplication of/in Image;+Investigation by Third Party;+Original Data and/or Images not Provided and/or not Available;+Paper Mill;', '10.1016/j.omtn.2019.12.014': '+Concerns/Issues About Image;+Euphemisms for Plagiarism;+Investigation by Journal/Publisher;+Investigation by Third Party;+Plagiarism of Image;', '10.1016/j.omtn.2019.12.025': '+Concerns/Issues About Data;+Concerns/Issues about Results and/or Conclusions;', '10.1016/j.biopha.2018.05.078': '+Author Unresponsive;+Concerns/Issues About Data;+Duplication of/in Image;+Investigation by Third Party;+Original Data and/or Images not Provided and/or not Available;+Paper Mill;+Upgrade/Update of Prior Notice(s);\n                                                                                                  +Concerns/Issues About Data;+Concerns/Is

In [ ]:
a = df["OriginalPaperDOI"].isin(['10.1016/j.lfs.2020.117519'])
print(df[a].empty)

In [ ]:
reason = []
for doi in retracted.keys():
    a = df['OriginalPaperDOI'].isin([doi])
    if not df[a].empty:
        print(doi)
        print(df[a]['Reason'].to_string(index = False))#.split(';'))
        reason.append(df[a]['Reason'].to_string(index = False))
            #break

In [47]:
pd.set_option('display.max_colwidth', None)
dico_df = pd.DataFrame(retracted.items(), columns=['DOI','Reasons'])
print(len(dico_df))
display(dico_df.head(2))
merged_df = my_df.merge(dico_df, on="DOI")
merged_df = merged_df.drop(['Section title','IMRD','Cited title'],axis = 1)
merged_df.rename(columns={'DOI': 'Retracted DOI'}, inplace=True)
display(merged_df)

26


,DOI,Reasons
0,10.1016/j.biopha.2018.03.122,+Duplication of/in Image;+Investigation by Third Party;+Paper Mill;+Results Not Reproducible;
1,10.1016/j.lfs.2020.117519,+Concerns/Issues About Data;+Duplication of/in Image;+Investigation by Third Party;+Original Data and/or Images not Provided and/or not Available;+Paper Mill;


,Citing DOI,Citation ID,CC paragraph,CC prev_next,Retracted DOI,Reasons
0,10.3390/cancers12113118,B46-cancers-12-03118,"Data on microRNAs, the sample source, the regulatory mode of microRNAs, the target genes/proteins of microRNAs, and their effect on PTC cells from the 137 studies were selected and presented in Table 1 [ 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 , 32 , 33 , 34 , 35 , 36 , 37 , 38 , 39 , 40 , 41 , 42 , 43 , 44 , 45 , 46 , 47 , 48 , 49 , 50 , 51 , 52 , 53 , 54 , 55 , 56 , 57 , 58 , 59 , 60 , 61 , 62 , 63 , 64 , 65 , 66 , 67 , 68 , 69 , 70 , 71 , 72 , 73 , 74 , 75 , 76 , 77 , 78 , 79 , 80 , 81 , 82 , 83 , 84 , 85 , 86 , 87 , 88 , 89 , 90 , 91 , 92 , 93 , 94 , 95 , 96 , 97 , 98 , 99 , 100 , 101 , 102 , 103 , 104 , 105 , 106 , 107 , 108 , 109 , 110 , 111 , 112 , 113 , 114 , 115 , 116 , 117 , 118 , 119 , 120 , 121 , 122 , 123 , 124 , 125 , 126 , 127 , 128 , 129 , 130 , 131 , 132 , 133 , 134 , 135 , 136 , 137 , 138 , 139 , 140 , 141 , 142 , 143 , 144 , 145 , 146 , 147 , 148 ].","Data on microRNAs, the sample source, the regulatory mode of microRNAs, the target genes/proteins of microRNAs, and their effect on PTC cells from the 137 studies were selected and presented in Table 1 [ 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 , 32 , 33 , 34 , 35 , 36 , 37 , 38 , 39 , 40 , 41 , 42 , 43 , 44 , 45 , 46 , 47 , 48 , 49 , 50 , 51 , 52 , 53 , 54 , 55 , 56 , 57 , 58 , 59 , 60 , 61 , 62 , 63 , 64 , 65 , 66 , 67 , 68 , 69 , 70 , 71 , 72 , 73 , 74 , 75 , 76 , 77 , 78 , 79 , 80 , 81 , 82 , 83 , 84 , 85 , 86 , 87 , 88 , 89 , 90 , 91 , 92 , 93 , 94 , 95 , 96 , 97 , 98 , 99 , 100 , 101 , 102 , 103 , 104 , 105 , 106 , 107 , 108 , 109 , 110 , 111 , 112 , 113 , 114 , 115 , 116 , 117 , 118 , 119 , 120 , 121 , 122 , 123 , 124 , 125 , 126 , 127 , 128 , 129 , 130 , 131 , 132 , 133 , 134 , 135 , 136 , 137 , 138 , 139 , 140 , 141 , 142 , 143 , 144 , 145 , 146 , 147 , 148 ].",10.1016/j.biopha.2018.03.122,+Duplication of/in Image;+Investigation by Third Party;+Paper Mill;+Results Not Reproducible;
1,10.3390/cancers12113118,B64-cancers-12-03118,"Data on microRNAs, the sample source, the regulatory mode of microRNAs, the target genes/proteins of microRNAs, and their effect on PTC cells from the 137 studies were selected and presented in Table 1 [ 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 , 32 , 33 , 34 , 35 , 36 , 37 , 38 , 39 , 40 , 41 , 42 , 43 , 44 , 45 , 46 , 47 , 48 , 49 , 50 , 51 , 52 , 53 , 54 , 55 , 56 , 57 , 58 , 59 , 60 , 61 , 62 , 63 , 64 , 65 , 66 , 67 , 68 , 69 , 70 , 71 , 72 , 73 , 74 , 75 , 76 , 77 , 78 , 79 , 80 , 81 , 82 , 83 , 84 , 85 , 86 , 87 , 88 , 89 , 90 , 91 , 92 , 93 , 94 , 95 , 96 , 97 , 98 , 99 , 100 , 101 , 102 , 103 , 104 , 105 , 106 , 107 , 108 , 109 , 110 , 111 , 112 , 113 , 114 , 115 , 116 , 117 , 118 , 119 , 120 , 121 , 122 , 123 , 124 , 125 , 126 , 127 , 128 , 129 , 130 , 131 , 132 , 133 , 134 , 135 , 136 , 137 , 138 , 139 , 140 , 141 , 142 , 143 , 144 , 145 , 146 , 147 , 148 ].","Data on microRNAs, the sample source, the regulatory mode of microRNAs, the target genes/proteins of microRNAs, and their effect on PTC cells from the 137 studies were selected and presented in Table 1 [ 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 , 32 , 33 , 34 , 35 , 36 , 37 , 38 , 39 , 40 , 41 , 42 , 43 , 44 , 45 , 46 , 47 , 48 , 49 , 50 , 51 , 52 , 53 , 54 , 55 , 56 , 57 , 58 , 59 , 60 , 61 , 62 , 63 , 64 , 65 , 66 , 67 , 68 , 69 , 70 , 71 , 72 , 73 , 74 , 75 , 76 , 77 , 78 , 79 , 80 , 81 , 82 , 83 , 84 , 85 , 86 , 87 , 88 , 89 , 90 , 91 , 92 , 93 , 94 , 95 , 96 , 97 , 98 , 99 , 100 , 101 , 102 , 103 , 104 , 105 , 106 , 107 , 108 , 109 , 110 , 111 , 112 , 113 , 114 , 115 , 116 , 117 , 118 , 119 , 120 , 121 , 122 , 123 , 124 , 125 , 126 , 127 , 128 , 129 , 130 , 131 , 132 , 133 , 134 , 135 , 136 , 137 , 138 , 139 , 140

In [49]:
merged_df.to_csv('Retracted_sample', sep='\t', index=False)